# The Spectral Theorem
#### By Clara & Roy

## Implementation in SageMath

In [1]:
from IPython.display import display, Math

def spectral_theorem(A, over_CC = True, polarform = False):
    # The next line only exists to prevent user errors.
    if A.apply_map(imag) != 0: over_CC = True
    
    # The next line checks if the matrix is normal, i.e. whether A*A = AA*
    if not A.is_normal():
        return "Sorry, this matrix is not normal, so the Spectral Theorem is not applicable.", ":("
    
    eigenvectors, eigenvalues = [], []
    
    # We save all eigenvectors and their corresponding eigenvalue:
    for i in A.eigenvectors_right():
        for j in i[1]:
            eigenvectors.append(j)
            eigenvalues.append(i[0])
    
    # If we operate on the complex field or if the matrix is symmetric, we can diagonalise it.
    if over_CC or A.is_symmetric():
        # To get an orthonormal basis of eigenvectors, we simply have to normalise our eigenvectors.
        for i in range(len(eigenvectors)):
            eigenvectors[i] /= eigenvectors[i].norm()
        D = diagonal_matrix(eigenvalues)
    else:
        # In this case, A is either orthogonal or only normal over the real field.
        # As such, we need to pick out the eigenvectors which have complex eigenvalues.
        real_basis, complex_vectors = [], []
        for i in range(len(eigenvalues)):
            if eigenvalues[i].imag() == 0:
                # Real eigenvalue, we only need to normalise the corresponding eigenvector.
                real_basis.append(eigenvectors[i] / eigenvectors[i].norm())
            else:
                # Complex eigenvalue!
                complex_vectors.append(eigenvectors[i])
        # A basis is formed by taking the real part and the imaginary part as basis vectors.
        # Since two eigenvalues are complex conjugated, we also check if our vectors is already in the list.
        for i in range(len(complex_vectors)):
            real_part, imag_part = complex_vectors[i].apply_map(real), complex_vectors[i].apply_map(imag)
            real_part /= real_part.norm()
            imag_part /= imag_part.norm()
            if real_part not in real_basis:
                real_basis.append(real_part)
                real_basis.append(imag_part)
        
        eigenvectors = real_basis
        # We will also reorder our list of eigenvalues, s.t. the real values are at the front.
        # This is done to construct our D.
        reorder, complex_entry = [], False
        for i in eigenvalues:
            if i.imag() == 0: reorder.append(i)
        for i in eigenvalues:
            if i not in reorder: reorder.append(i)
            
        D_list = [[0 for j in range(A.ncols())] for i in range(A.nrows())]
        
        # If the matrix is unitary, we can also use the polar form.
        if polarform and A.is_unitary():
            for i in range(len(reorder)):
                if reorder[i].imag() == 0: D_list[i][i] = reorder[i]
                else:
                    arg = arctan2(reorder[i].imag(), reorder[i].real())
                    if not complex_entry:
                        D_list[i][i] = cos(arg)
                        D_list[i][i+1] = sin(arg)
                        complex_entry = True
                    else:
                        D_list[i][i] = cos(arg)
                        D_list[i][i-1] = sin(arg)
                        complex_entry = False
            D = matrix(D_list)
        # If we want to represent the entries as usual.
        else:
            for i in range(len(reorder)):
                if reorder[i].imag() == 0: D_list[i][i] = reorder[i]
                else:
                    if not complex_entry:
                        D_list[i][i] = reorder[i].real()
                        D_list[i][i+1] = reorder[i].imag()
                        complex_entry = True
                    else:
                        D_list[i][i] = reorder[i].real()
                        D_list[i][i-1] = reorder[i].imag()
                        complex_entry = False
            D = matrix(D_list)

    P = matrix(eigenvectors).transpose()
    return P, D

## Lots of examples!

In [21]:
# Hermitian matrix.
A = matrix(SR, [[3, 1 - 2*I], [1 + 2*I, -1]])
show("A = ", A)

P, D = spectral_theorem(A)
show("P = ", P)
show("D = ", D)
display(Math(r"P \cdot D \cdot P^* ="))
show(P * D * P.conjugate_transpose())
display(Math(r"P^* \cdot A \cdot P ="))
show(P.conjugate_transpose() * A * P)

'A = ' [       3 -2*I + 1]
[ 2*I + 1       -1]

'P = ' [              1/sqrt(6)             1/sqrt(6/5)]
[     -(2*I + 1)/sqrt(6) (2/5*I + 1/5)/sqrt(6/5)]

'D = ' [-2  0]
[ 0  4]

<IPython.core.display.Math object>

[       3 -2*I + 1]
[ 2*I + 1       -1]

<IPython.core.display.Math object>

[-2  0]
[ 0  4]

In [20]:
# Unitary matrix over the complex field.
A = matrix(CDF, [[1/sqrt(2), 1/sqrt(2)], [I/sqrt(2), -I/sqrt(2)]])
show("A = ", A)

P, D = spectral_theorem(A)
show("P = ", P)
show("D = ", D)
display(Math(r"P \cdot D \cdot P^* ="))
show(P * D * P.conjugate_transpose())
display(Math(r"P^* \cdot A \cdot P ="))
show(P.conjugate_transpose() * A * P)

'A = ' [   0.7071067811865475    0.7071067811865475]
[ 0.7071067811865475*I -0.7071067811865475*I]

'P = ' [                         0.8880738339771154 -0.3250575836718681 + 0.32505758367186816*I]
[  0.3250575836718681 + 0.3250575836718682*I                          0.8880738339771155]

'D = ' [  0.965925826289068 + 0.25881904510252063*I                                         0.0]
[                                        0.0 -0.25881904510252074 - 0.9659258262890682*I]

<IPython.core.display.Math object>

[ 0.7071067811865475 - 1.0157783639972121e-17*I   0.7071067811865475 - 6.096868888106627e-17*I]
[-1.1679078044036417e-16 + 0.7071067811865476*I  -1.208940397706737e-16 - 0.7071067811865478*I]

<IPython.core.display.Math object>

[        0.9659258262890683 + 0.2588190451025208*I   1.068904066940029e-16 + 3.321311326543735e-17*I]
[1.3900163400470237e-16 + 1.5511647439008722e-16*I       -0.25881904510252085 - 0.9659258262890685*I]

In [19]:
# Symmetric matrix.
A = matrix(QQ, [[1, 7, 3], [7, 4, 5], [3, 5, 2]])
show("A = ", A)

P, D = spectral_theorem(A)
show("P = ", P)
show("D = ", D)
display(Math(r"P \cdot D \cdot P^t ="))
show(P * D * P.conjugate_transpose())
display(Math(r"P^t \cdot A \cdot P ="))
show(P.conjugate_transpose() * A * P)

'A = ' [1 7 3]
[7 4 5]
[3 5 2]

'P = ' [ 0.7190661678538667?  0.4419300493194164?  0.5363223636550065?]
[-0.6720730235880483?  0.2458718069651456?   0.698473267566421?]
[ 0.1768097769162479? -0.8626962884168637?  0.4738073624782483?]

'D = ' [ -4.804864727297083?                    0                    0]
[                   0 -0.9618177533520375?                    0]
[                   0                    0   12.76668248064912?]

<IPython.core.display.Math object>

[1.000000000000000? 7.000000000000000? 3.000000000000000?]
[7.000000000000000? 4.000000000000000? 5.000000000000000?]
[3.000000000000000? 5.000000000000000? 2.000000000000000?]

<IPython.core.display.Math object>

[ -4.804864727297083?              0.?e-16              0.?e-16]
[             0.?e-17 -0.9618177533520375?              0.?e-17]
[             0.?e-16              0.?e-16   12.76668248064912?]

In [18]:
# Normal matrix over the real field.
A = matrix(SR, [[2, 0, 3, 1], [0, 2, 1, 3], [-3, -1, 2, 0], [-1, -3, 0, 2]])
show("A = ", A)

P, D = spectral_theorem(A, False)
show("P = ", P)
show("D = ", D)
display(Math(r"P \cdot D \cdot P^t ="))
show(P * D * P.conjugate_transpose())
display(Math(r"P^t \cdot A \cdot P ="))
show(P.conjugate_transpose() * A * P)

'A = ' [ 2  0  3  1]
[ 0  2  1  3]
[-3 -1  2  0]
[-1 -3  0  2]

'P = ' [ 1/2*sqrt(2)            0  1/2*sqrt(2)            0]
[ 1/2*sqrt(2)            0 -1/2*sqrt(2)            0]
[           0 -1/2*sqrt(2)            0 -1/2*sqrt(2)]
[           0 -1/2*sqrt(2)            0  1/2*sqrt(2)]

'D = ' [ 2 -4  0  0]
[ 4  2  0  0]
[ 0  0  2 -2]
[ 0  0  2  2]

<IPython.core.display.Math object>

[ 2  0  3  1]
[ 0  2  1  3]
[-3 -1  2  0]
[-1 -3  0  2]

<IPython.core.display.Math object>

[ 2 -4  0  0]
[ 4  2  0  0]
[ 0  0  2 -2]
[ 0  0  2  2]

In [17]:
# Orthogonal matrix.
A = matrix(SR, [[0, 0, 0, 1], [0, 0, 1, 0], [1, 0, 0, 0], [0, 1, 0, 0]])
show("A = ", A)

P, D = spectral_theorem(A, False)
show("P = ", P)
show("D = ", D)
display(Math(r"P \cdot D \cdot P^* ="))
show(P * D * P.conjugate_transpose())
display(Math(r"P^* \cdot A \cdot P ="))
show(P.conjugate_transpose() * A * P)

'A = ' [0 0 0 1]
[0 0 1 0]
[1 0 0 0]
[0 1 0 0]

'P = ' [         1/2          1/2  1/2*sqrt(2)            0]
[         1/2          1/2 -1/2*sqrt(2)            0]
[        -1/2          1/2            0 -1/2*sqrt(2)]
[        -1/2          1/2            0  1/2*sqrt(2)]

'D = ' [-1  0  0  0]
[ 0  1  0  0]
[ 0  0  0  1]
[ 0  0 -1  0]

<IPython.core.display.Math object>

[0 0 0 1]
[0 0 1 0]
[1 0 0 0]
[0 1 0 0]

<IPython.core.display.Math object>

[-1  0  0  0]
[ 0  1  0  0]
[ 0  0  0  1]
[ 0  0 -1  0]

In [16]:
# Orthogonal matrix over the real field (with polar form).
A = matrix(SR, [[1/3, -2/3, 2/3], [2/3, -1/3, -2/3], [2/3, 2/3, 1/3]])
show("A = ", A)

P, D = spectral_theorem(A, False, True)
show("P = ", P)
show("D = ", D)
display(Math(r"P \cdot D \cdot P^t ="))
show(P * D * P.conjugate_transpose())
display(Math(r"P^t \cdot A \cdot P ="))
show(P.conjugate_transpose() * A * P)

'A = ' [ 1/3 -2/3  2/3]
[ 2/3 -1/3 -2/3]
[ 2/3  2/3  1/3]

'P = ' [ 1/2*sqrt(2)  1/2*sqrt(2)            0]
[           0            0            1]
[ 1/2*sqrt(2) -1/2*sqrt(2)            0]

'D = ' [                           1                            0                            0]
[                           0 cos(-pi + arctan(2*sqrt(2))) sin(-pi + arctan(2*sqrt(2)))]
[                           0  sin(pi - arctan(2*sqrt(2)))  cos(pi - arctan(2*sqrt(2)))]

<IPython.core.display.Math object>

[   1/2*cos(-pi + arctan(2*sqrt(2))) + 1/2  1/2*sqrt(2)*sin(-pi + arctan(2*sqrt(2)))   -1/2*cos(-pi + arctan(2*sqrt(2))) + 1/2]
[  1/2*sqrt(2)*sin(pi - arctan(2*sqrt(2)))               cos(pi - arctan(2*sqrt(2)))  -1/2*sqrt(2)*sin(pi - arctan(2*sqrt(2)))]
[  -1/2*cos(-pi + arctan(2*sqrt(2))) + 1/2 -1/2*sqrt(2)*sin(-pi + arctan(2*sqrt(2)))    1/2*cos(-pi + arctan(2*sqrt(2))) + 1/2]

<IPython.core.display.Math object>

[           1            0            0]
[           0         -1/3 -2/3*sqrt(2)]
[           0  2/3*sqrt(2)         -1/3]